In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from functools import partial

class RGCNLayer(nn.Module):
    def __init__(self, in_feat, out_feat, num_rels, num_bases=-1, bias=None,
                 activation=None, is_input_layer=False):
        super(RGCNLayer, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.bias = bias
        self.activation = activation
        self.is_input_layer = is_input_layer

        # sanity check
        if self.num_bases <= 0 or self.num_bases > self.num_rels:
            self.num_bases = self.num_rels

        # weight bases in equation (3)
        self.weight = nn.Parameter(torch.Tensor(self.num_bases, self.in_feat,
                                                self.out_feat))
        if self.num_bases < self.num_rels:
            # linear combination coefficients in equation (3)
            self.w_comp = nn.Parameter(torch.Tensor(self.num_rels, self.num_bases))

        # add bias
        if self.bias:
            self.bias = nn.Parameter(torch.Tensor(out_feat))

        # init trainable parameters
        nn.init.xavier_uniform_(self.weight,
                                gain=nn.init.calculate_gain('relu'))
        if self.num_bases < self.num_rels:
            nn.init.xavier_uniform_(self.w_comp,
                                    gain=nn.init.calculate_gain('relu'))
        if self.bias:
            nn.init.xavier_uniform_(self.bias,
                                    gain=nn.init.calculate_gain('relu'))

    def forward(self, g):
        if self.num_bases < self.num_rels:
            # generate all weights from bases (equation (3))
            weight = self.weight.view(self.in_feat, self.num_bases, self.out_feat)
            weight = torch.matmul(self.w_comp, weight).view(self.num_rels,
                                                        self.in_feat, self.out_feat)
        else:
            weight = self.weight

        if self.is_input_layer:
            def message_func(edges):
                # for input layer, matrix multiply can be converted to be
                # an embedding lookup using source node id
                # embed = weight.view(-1, self.out_feat)
                # index = edges.data['rel_type'] * self.in_feat + edges.src['id']
                # index = edges.data['rel_type'] * self.in_feat + edges.src['id']
                # return {'msg': embed[index] * edges.data['norm']}
                w = weight[edges.data['rel_type']]
                msg = torch.bmm(edges.src['init_h'].unsqueeze(1), w).squeeze()
                msg = msg * edges.data['norm']
                return {'msg': msg}                
        else:
            def message_func(edges):
                w = weight[edges.data['rel_type']]
                msg = torch.bmm(edges.src['h'].unsqueeze(1), w).squeeze()
                msg = msg * edges.data['norm']
                return {'msg': msg}

        def apply_func(nodes):
            h = nodes.data['h']
            if self.bias:
                h = h + self.bias
            if self.activation:
                h = self.activation(h)
            return {'h': h}

        g.update_all(message_func, fn.sum(msg='msg', out='h'), apply_func)

Define full R-GCN model
~~~~~~~~~~~~~~~~~~~~~~~



In [3]:
class Model(nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels,node_feature_array,
                 num_bases=-1, num_hidden_layers=1):
        super(Model, self).__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels
        self.node_feature_array = node_feature_array
        self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers
        

        # create rgcn layers
        self.build_model()

        # create initial features
        self.features = self.create_features()

    def build_model(self):
        self.layers = nn.ModuleList()
        # input to hidden
        i2h = self.build_input_layer()
        self.layers.append(i2h)
        # hidden to hidden
        for idx in range(self.num_hidden_layers - 1):
            h2h = self.build_hidden_layer(idx)
            self.layers.append(h2h)
        # hidden to output
        h2o = self.build_output_layer()
        self.layers.append(h2o)

    # initialize feature for each node
    def create_features(self):
        # features = torch.arange(self.num_nodes)
        features = torch.from_numpy(self.node_feature_array)
        return features

    def build_input_layer(self):
        return RGCNLayer(self.node_feature_array.shape[1], self.h_dim[0], self.num_rels, self.num_bases,
                         activation=F.relu, is_input_layer=True)

    def build_hidden_layer(self,idx):
        return RGCNLayer(self.h_dim[idx], self.h_dim[idx+1], self.num_rels, self.num_bases,
                         activation=F.relu)

    def build_output_layer(self):
        return RGCNLayer(self.h_dim[-1], self.out_dim, self.num_rels, self.num_bases,
                         activation=partial(F.softmax, dim=1))

    def forward(self, g):
        if self.features is not None:
            # g.ndata['id'] = self.features
            g.ndata['init_h'] = self.features
        for layer in self.layers:
            layer(g)
        return g.ndata.pop('h')

Handle dataset
~~~~~~~~~~~~~~~~
In this tutorial, we use AIFB dataset from R-GCN paper:



In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [5]:
def get_dist(df,col):
    df_cnt = df.groupby([col]+['etype'])['time'].count().unstack(1,fill_value=0)
    df_dist = pd.DataFrame(df_cnt.values / df_cnt.sum(1).values.reshape(-1,1),
                                               columns=df_cnt.columns,
                                               index=df_cnt.index)
    return df_dist

In [6]:
from sklearn.preprocessing import LabelEncoder

alpha_network = pd.read_csv('raw_data/alpha/alpha_network.csv',header=None)
alpha_network.columns = ['src_raw','dst_raw','etype','time']
etype_encoder = LabelEncoder()
alpha_network['etype'] = etype_encoder.fit_transform(alpha_network.etype)
alpha_gt = pd.read_csv('raw_data/alpha/alpha_gt.csv',header=None)
alpha_gt.columns = ['node_id_raw','label']
alpha_gt = alpha_gt.drop_duplicates('node_id_raw')

# edge_normの計算
alpha_src_cnt = alpha_network.groupby(['src_raw','etype'])['time'].count().unstack(1,fill_value=0)

alpha_src_dist = pd.DataFrame(alpha_src_cnt.values/alpha_src_cnt.sum(1).values.reshape(-1,1),
                                                        index=alpha_src_cnt.index,
                                                        columns=alpha_src_cnt.columns)

merged_network = pd.merge(alpha_network,alpha_src_dist.stack().reset_index(),on=['src_raw','etype'])


label_encoder = LabelEncoder()
label_encoder.fit(np.hstack((merged_network.src_raw,
                                                   merged_network.dst_raw,
                                                   alpha_gt.node_id_raw)))

merged_network['src'] = label_encoder.transform(merged_network.src_raw)

merged_network['dst'] = label_encoder.transform(merged_network.dst_raw)

alpha_gt['node_id'] = label_encoder.transform(alpha_gt.node_id_raw)
alpha_gt['label'] = alpha_gt['label'].map(lambda x:1 if x==-1 else 0)

# padding
alpha_gt_padded = pd.merge(pd.DataFrame(np.arange(label_encoder.classes_.shape[0])),alpha_gt,
                                      left_index=True,right_on='node_id',how='left').fillna(0.5).sort_values('node_id')

num_nodes = label_encoder.classes_.shape[0]
num_rels = merged_network.etype.unique().shape[0]
num_classes = alpha_gt.label.unique().shape[0]
labels = alpha_gt_padded['label'].values.astype(int).reshape(-1,1)
all_idx = alpha_gt['node_id'].values

# edge type and normalization factor
edge_type = torch.from_numpy(merged_network['etype'].values)
edge_norm = torch.from_numpy(merged_network[0].values.astype('float32')).unsqueeze(1)

labels = torch.from_numpy(labels).view(-1)

node_feature_df = pd.concat([get_dist(merged_network,'src'),get_dist(merged_network,'dst')],1).fillna(0).sort_index()
node_feature_array = node_feature_df.values.astype('float32')

known_labels = alpha_gt['label'].values

Create graph and model
~~~~~~~~~~~~~~~~~~~~~~~



In [7]:
# configurations
n_hidden = [32,16] # number of hidden units
n_bases = -1 # -1 # use number of relations as number of bases
n_hidden_layers = 2 # use 1 input layer, 1 output layer, no hidden layer
n_epochs = 300 # epochs to train
lr = 0.01 # learning rate
l2norm = 0.0001 # L2 norm coefficient

## cross validation

In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
kf = StratifiedKFold(n_splits=10)

In [10]:
auc_scores = []

for i, (for_train_val_idx, for_test_idx) in enumerate(kf.split(np.arange(len(all_idx)),y=known_labels)):
    train_val_idx = all_idx[for_train_val_idx]
    train_idx, val_idx = train_test_split(train_val_idx,test_size=0.33,stratify=known_labels[for_train_val_idx])
    test_idx = all_idx[for_test_idx]
    # create graph
    g = DGLGraph()
    g.add_nodes(num_nodes)
    g.add_edges(merged_network['src'].values, merged_network['dst'].values)
    g.edata.update({'rel_type': edge_type, 'norm': edge_norm})
    # create model
    model = Model(len(g),
                  n_hidden,
                  num_classes,
                  num_rels,
                  node_feature_array,
                  num_bases=n_bases,
                  num_hidden_layers=n_hidden_layers)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)
    print("Start {}-th fold".format(i))
    print("==== Train Phase ====")
    model.train()
    best_auc = 0.0
    best_auc_logits = None
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        logits = model.forward(g)
        loss = F.cross_entropy(logits[train_idx], labels[train_idx])
        loss.backward()

        optimizer.step()

        train_auc = roc_auc_score(y_true=labels[train_idx].detach().numpy(),y_score=logits[train_idx].detach().numpy()[:,1])
        train_loss = F.cross_entropy(logits[train_idx], labels[train_idx])
        val_auc = roc_auc_score(y_true=labels[val_idx].detach().numpy(),y_score=logits[val_idx].detach().numpy()[:,1])
        val_loss = F.cross_entropy(logits[val_idx], labels[val_idx])
        
        if val_auc >= best_auc:
            best_auc = val_auc
            best_auc_logits = logits
            
        print("Epoch {:05d} | ".format(epoch) +
              "Train AUC: {:.4f} | Train Loss: {:.4f} | ".format(
                  train_auc, loss.item()) +
              "Validation AUC: {:.4f} | Validation loss: {:.4f}".format(
                  val_auc, val_loss.item()))
    print("==== Test Phase ====")
    model.eval()
    test_auc = roc_auc_score(y_true=labels[test_idx].detach().numpy(),y_score=best_auc_logits[test_idx].detach().numpy()[:,1])
    auc_scores.append(test_auc)
    print("test auc : {}".format(test_auc))
    print("=================")

Start 0-th fold
==== Train Phase ====


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/dgl/frame.py:204: UserWarning: Initializer is not set. Use zero initializer instead. To suppress this warning, use `set_initializer` to explicitly specify which initializer to use.
  dgl_warning('Initializer is not set. Use zero initializer instead.'


Epoch 00000 | Train AUC: 0.2319 | Train Loss: 0.8954 | Validation AUC: 0.2842 | Validation loss: 0.8627
Epoch 00001 | Train AUC: 0.8344 | Train Loss: 0.5357 | Validation AUC: 0.8475 | Validation loss: 0.5171
Epoch 00002 | Train AUC: 0.6549 | Train Loss: 0.6968 | Validation AUC: 0.7583 | Validation loss: 0.6112
Epoch 00003 | Train AUC: 0.8555 | Train Loss: 0.5260 | Validation AUC: 0.8633 | Validation loss: 0.4876
Epoch 00004 | Train AUC: 0.8503 | Train Loss: 0.5326 | Validation AUC: 0.8658 | Validation loss: 0.4876
Epoch 00005 | Train AUC: 0.8438 | Train Loss: 0.5282 | Validation AUC: 0.8538 | Validation loss: 0.5028
Epoch 00006 | Train AUC: 0.8403 | Train Loss: 0.5275 | Validation AUC: 0.8400 | Validation loss: 0.5181
Epoch 00007 | Train AUC: 0.8380 | Train Loss: 0.5280 | Validation AUC: 0.8375 | Validation loss: 0.5237
Epoch 00008 | Train AUC: 0.8355 | Train Loss: 0.5270 | Validation AUC: 0.8383 | Validation loss: 0.5254
Epoch 00009 | Train AUC: 0.8284 | Train Loss: 0.5241 | Validatio

Epoch 00079 | Train AUC: 0.9251 | Train Loss: 0.4039 | Validation AUC: 0.9258 | Validation loss: 0.4088
Epoch 00080 | Train AUC: 0.9261 | Train Loss: 0.4118 | Validation AUC: 0.9225 | Validation loss: 0.4224
Epoch 00081 | Train AUC: 0.9247 | Train Loss: 0.4182 | Validation AUC: 0.9133 | Validation loss: 0.4225
Epoch 00082 | Train AUC: 0.9237 | Train Loss: 0.4272 | Validation AUC: 0.9133 | Validation loss: 0.4222
Epoch 00083 | Train AUC: 0.9161 | Train Loss: 0.4277 | Validation AUC: 0.9125 | Validation loss: 0.4220
Epoch 00084 | Train AUC: 0.9153 | Train Loss: 0.4278 | Validation AUC: 0.9125 | Validation loss: 0.4218
Epoch 00085 | Train AUC: 0.9156 | Train Loss: 0.4276 | Validation AUC: 0.9012 | Validation loss: 0.4216
Epoch 00086 | Train AUC: 0.9159 | Train Loss: 0.4271 | Validation AUC: 0.9012 | Validation loss: 0.4214
Epoch 00087 | Train AUC: 0.9151 | Train Loss: 0.4266 | Validation AUC: 0.8892 | Validation loss: 0.4213
Epoch 00088 | Train AUC: 0.9147 | Train Loss: 0.4263 | Validatio

Epoch 00158 | Train AUC: 0.9081 | Train Loss: 0.4259 | Validation AUC: 0.8867 | Validation loss: 0.4198
Epoch 00159 | Train AUC: 0.9081 | Train Loss: 0.4259 | Validation AUC: 0.8867 | Validation loss: 0.4198
Epoch 00160 | Train AUC: 0.9081 | Train Loss: 0.4259 | Validation AUC: 0.8867 | Validation loss: 0.4198
Epoch 00161 | Train AUC: 0.9081 | Train Loss: 0.4259 | Validation AUC: 0.8867 | Validation loss: 0.4198
Epoch 00162 | Train AUC: 0.9081 | Train Loss: 0.4259 | Validation AUC: 0.8867 | Validation loss: 0.4198
Epoch 00163 | Train AUC: 0.9081 | Train Loss: 0.4259 | Validation AUC: 0.8867 | Validation loss: 0.4198
Epoch 00164 | Train AUC: 0.9081 | Train Loss: 0.4259 | Validation AUC: 0.8867 | Validation loss: 0.4198
Epoch 00165 | Train AUC: 0.9081 | Train Loss: 0.4259 | Validation AUC: 0.8858 | Validation loss: 0.4198
Epoch 00166 | Train AUC: 0.9081 | Train Loss: 0.4259 | Validation AUC: 0.8858 | Validation loss: 0.4198
Epoch 00167 | Train AUC: 0.9073 | Train Loss: 0.4259 | Validatio

...

Epoch 00237 | Train AUC: 0.9279 | Train Loss: 0.3696 | Validation AUC: 0.9282 | Validation loss: 0.4173
Epoch 00238 | Train AUC: 0.9279 | Train Loss: 0.3696 | Validation AUC: 0.9282 | Validation loss: 0.4174
Epoch 00239 | Train AUC: 0.9279 | Train Loss: 0.3696 | Validation AUC: 0.9282 | Validation loss: 0.4175
Epoch 00240 | Train AUC: 0.9279 | Train Loss: 0.3696 | Validation AUC: 0.9282 | Validation loss: 0.4176
Epoch 00241 | Train AUC: 0.9279 | Train Loss: 0.3696 | Validation AUC: 0.9282 | Validation loss: 0.4177
Epoch 00242 | Train AUC: 0.9279 | Train Loss: 0.3696 | Validation AUC: 0.9282 | Validation loss: 0.4178
Epoch 00243 | Train AUC: 0.9272 | Train Loss: 0.3696 | Validation AUC: 0.9282 | Validation loss: 0.4179
Epoch 00244 | Train AUC: 0.9272 | Train Loss: 0.3696 | Validation AUC: 0.9282 | Validation loss: 0.4181
Epoch 00245 | Train AUC: 0.9272 | Train Loss: 0.3696 | Validation AUC: 0.9282 | Validation loss: 0.4182
Epoch 00246 | Train AUC: 0.9272 | Train Loss: 0.3696 | Validatio

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/dgl/frame.py:204: UserWarning: Initializer is not set. Use zero initializer instead. To suppress this warning, use `set_initializer` to explicitly specify which initializer to use.
  dgl_warning('Initializer is not set. Use zero initializer instead.'


Epoch 00000 | Train AUC: 0.8053 | Train Loss: 0.5671 | Validation AUC: 0.8460 | Validation loss: 0.5231
Epoch 00001 | Train AUC: 0.8436 | Train Loss: 0.5499 | Validation AUC: 0.8520 | Validation loss: 0.5393
Epoch 00002 | Train AUC: 0.8545 | Train Loss: 0.5274 | Validation AUC: 0.8609 | Validation loss: 0.5238
Epoch 00003 | Train AUC: 0.8636 | Train Loss: 0.5116 | Validation AUC: 0.8560 | Validation loss: 0.5137
Epoch 00004 | Train AUC: 0.8723 | Train Loss: 0.5004 | Validation AUC: 0.8669 | Validation loss: 0.4991
Epoch 00005 | Train AUC: 0.8877 | Train Loss: 0.4824 | Validation AUC: 0.8831 | Validation loss: 0.4632
Epoch 00006 | Train AUC: 0.9132 | Train Loss: 0.4710 | Validation AUC: 0.9008 | Validation loss: 0.4195
Epoch 00007 | Train AUC: 0.9569 | Train Loss: 0.3972 | Validation AUC: 0.9460 | Validation loss: 0.3920
Epoch 00008 | Train AUC: 0.6589 | Train Loss: 0.6891 | Validation AUC: 0.6190 | Validation loss: 0.7251
Epoch 00009 | Train AUC: 0.5872 | Train Loss: 0.7287 | Validatio

Epoch 00079 | Train AUC: 0.9658 | Train Loss: 0.3498 | Validation AUC: 0.9734 | Validation loss: 0.3414
Epoch 00080 | Train AUC: 0.9678 | Train Loss: 0.3538 | Validation AUC: 0.9685 | Validation loss: 0.3526
Epoch 00081 | Train AUC: 0.9682 | Train Loss: 0.3484 | Validation AUC: 0.9649 | Validation loss: 0.3555
Epoch 00082 | Train AUC: 0.9690 | Train Loss: 0.3485 | Validation AUC: 0.9565 | Validation loss: 0.3555
Epoch 00083 | Train AUC: 0.9688 | Train Loss: 0.3555 | Validation AUC: 0.9444 | Validation loss: 0.3555
Epoch 00084 | Train AUC: 0.9674 | Train Loss: 0.3556 | Validation AUC: 0.9403 | Validation loss: 0.3555
Epoch 00085 | Train AUC: 0.9666 | Train Loss: 0.3557 | Validation AUC: 0.9310 | Validation loss: 0.3555
Epoch 00086 | Train AUC: 0.9658 | Train Loss: 0.3559 | Validation AUC: 0.9310 | Validation loss: 0.3685
Epoch 00087 | Train AUC: 0.9654 | Train Loss: 0.3564 | Validation AUC: 0.9302 | Validation loss: 0.3696
Epoch 00088 | Train AUC: 0.9656 | Train Loss: 0.3560 | Validatio

KeyboardInterrupt: 

In [ ]:
np.mean(auc_scores)

結果

- (32,16)
    - amazon :
    - alpha : 
    - otc : 

In [ ]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)

print("start training...")
model.train()
for epoch in range(n_epochs):
    optimizer.zero_grad()
    logits = model.forward(g)
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])
    loss.backward()

    optimizer.step()

    train_acc = torch.sum(logits[train_idx].argmax(dim=1) == labels[train_idx])
    train_acc = train_acc.item() / len(train_idx)
    val_loss = F.cross_entropy(logits[val_idx], labels[val_idx])
    val_acc = torch.sum(logits[val_idx].argmax(dim=1) == labels[val_idx])
    val_acc = val_acc.item() / len(val_idx)
    print("Epoch {:05d} | ".format(epoch) +
          "Train Accuracy: {:.4f} | Train Loss: {:.4f} | ".format(
              train_acc, loss.item()) +
          "Validation Accuracy: {:.4f} | Validation loss: {:.4f}".format(
              val_acc, val_loss.item()))

In [ ]:
y_pred = logits[val_idx].detach().numpy()[:,1]

y_true = labels[val_idx].detach().numpy()

In [ ]:
roc_auc_score(y_true=labels[val_idx].detach().numpy(),y_score=logits[val_idx].detach().numpy()[:,1])

In [ ]:
_ = plt.hist(logits[val_idx].detach().numpy()[:,1][y_true==1],alpha=0.5)
_ = plt.hist(logits[val_idx].detach().numpy()[:,1][y_true==0],alpha=0.5)